Haal gebruikers op die films hebben gerate. Er ontstaat een RDD[(userId, movieId, rating)].

In [11]:
userdata = sc.textFile("../../../schooldata/nieuw/ratings_train.csv")

firstline = userdata.first()

data_normalized = userdata.filter(lambda x: x != firstline)

data_normalized = data_normalized.map(lambda x: x.split("::"))

user_mov_rating = data_normalized.map(lambda x:(int(x[0]), int(x[1]), float(x[2]))).cache()

user_mov_rating.take(5)

[(11973, 11, 3.0),
 (11973, 14, 4.0),
 (11973, 32, 2.0),
 (11973, 36, 3.0),
 (11973, 110, 5.0)]

Haal films op uit de database en map ze in een RDD zodat er een RDD[(movieId, title)] ontstaat.

In [14]:
moviedata = sc.textFile("../../../schooldata/nieuw/movies.csv")

firstline = moviedata.first()
movies_normalized = moviedata.filter(lambda x: x != firstline)\
.map(lambda x: x.split(','))

mov_id_name = movies_normalized.map(lambda x: (int(x[0]), x[1])).cache()

mov_id_name.take(10)

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)'),
 (6, 'Heat (1995)'),
 (7, 'Sabrina (1995)'),
 (8, 'Tom and Huck (1995)'),
 (9, 'Sudden Death (1995)'),
 (10, 'GoldenEye (1995)')]

Initialiseer de benodigde ALS parameters en laat ALS trainen. Vervolgens een voorspelde lijst in predictions stoppen, die we laten joinen met de test data split.

In [12]:
from pyspark.mllib.recommendation import ALS
import math

# verdeel rdd data 60% cls, 20% validatie en 20% tests
cls, validation, test = user_mov_rating.randomSplit([6, 2, 2], seed=None)

rank = None

# laat ALS op het cls rdd trainen.
trained_prototype = ALS.train(cls, rank = 8, seed = 5, iterations = 10, lambda_ = 0.01)

test_predictions = test.map(lambda x: (x[0], x[1]))

# de rating wordt voorspeld voor userId en movieId door het getrainde ALS prototype 
# en word gemapt als ((userId, movieId), rating).
predictions = trained_prototype.predictAll(test_predictions)\
.map(lambda x: ((x[0], x[1]), x[2]))

# de test rdd joint de predictions rdd en dan krijgen we ((userId, movieId), (rating, rating)).
rnp = test.map(lambda x: ((int(x[0]), int(x[1])), float(x[2])))\
.join(predictions)

computable_rnp = rnp.map(lambda x: (x[1][0] - x[1][1])**2)

# bereken de Root Mean Square Error bij de rnp data.
error = math.sqrt(computable_rnp.mean())

print('Met rank %s is de RMSE: %s' % (rank, error))

Met rank None is de RMSE: 0.8198104062684609


In [ ]:
predictions.filter(lambda x: x[1] > 4).takeOrdered(20, key=lambda x: x[0][0])

In [33]:
movieId_rating = user_mov_rating.map(lambda x: (x[1], x[2])).groupByKey()

movieId = mov_id_name.map(lambda x: x[0])

# map (movieId, (len(ratings), avg_rating)).
# avg_rating krijgen we door alle bijbehorende ratings op te sommen en 
# die te delen door de totale hoeveelheid (count) bijbehorende ratings
# en dat casten we naar een float.
# *met bijbehorende bedoelen we die bij de movieId horen
movieId_avgRating = movieId_rating\
.map(lambda x: (x[0], (len(x[1]), round(float(sum(i for i in x[1])) / len(x[1]), 1))))\
.filter(lambda x: x[1][1] == 4 or x[1][1] == 4.5 or x[1][1] == 5)

"""
n = mov_id_name.map(lambda x: x[0]).collect()
mi = movieId.collect()

user_no_rating = sc.parallelize([m for m in n if m in mi])
predictions = trained_prototype.predictAll(user_no_rating.map(lambda x: (0, x))).collect()
recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:50]

for i in range(len(recommendations)):
    print ("%2d: %s" % (i + 1, user_mov_rating[recommendations[i][1]])).encode('ascii', 'ignore')

"""
movieId_avgRating.take(50)

[(17, (20340, 4.0)),
 (98328, (1, 4.5)),
 (127007, (1, 4.5)),
 (98464, (1, 4.0)),
 (118133, (1, 4.5)),
 (118983, (1, 4.0)),
 (127228, (1, 4.0)),
 (102765, (1, 4.0)),
 (65909, (1, 4.0)),
 (119204, (1, 4.0)),
 (103003, (1, 4.0)),
 (103071, (1, 4.0)),
 (86717, (4, 4.0)),
 (107202, (1, 4.5)),
 (94928, (2, 4.0)),
 (103139, (1, 4.0)),
 (119663, (1, 4.0)),
 (33779, (203, 4.0)),
 (123981, (1, 4.5)),
 (119901, (1, 4.0)),
 (115821, (1, 4.0)),
 (99450, (1, 5.0)),
 (5304, (475, 4.0)),
 (1258, (23237, 4.0)),
 (1292, (6322, 4.0)),
 (128316, (1, 4.0)),
 (62781, (3, 4.0)),
 (54689, (1, 4.0)),
 (79458, (3, 4.0)),
 (99960, (5, 4.0)),
 (79594, (18, 4.0)),
 (100147, (1, 4.0)),
 (7140, (5, 4.0)),
 (92004, (1, 4.0)),
 (100266, (1, 4.5)),
 (96594, (1, 4.0)),
 (129013, (3, 4.0)),
 (79866, (1, 5.0)),
 (6273, (386, 4.0)),
 (100521, (1, 5.0)),
 (92412, (2, 4.0)),
 (26962, (3, 4.0)),
 (100725, (2, 4.0)),
 (108936, (1, 4.0)),
 (121227, (6, 4.0)),
 (129421, (1, 4.5)),
 (100946, (4, 4.0)),
 (117470, (1, 4.0)),
 (278

Werk het movie aanbevelingsalgoritme uit, zodat users die bepaalde movies nog niet gezien hebben, maar er wel in geinteresseerd kunnen zijn die films ook aanbevolen krijgen. Het gaat hierbij om movies met rating >= 4.